In [196]:
import cv2
import numpy as np
import os

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from OneNN import PM1NN
from Karcher_Mean import KarcherLSR

ImportError: cannot import name 'model_selection' from 'sklearn' (C:\Users\sahil\anaconda3\lib\site-packages\sklearn\__init__.py)

In [ ]:
def import_traffic(numFrames=35,height=50,width=50):
    """
    Collects data from /data/ folder in current working directory, with folder names as labels.
    Inputs:
    -- numFrames=25
    -- height=20,
    -- width=20
    Outputs:
    -- Data as a numpy array (vid number,frame,height,width,color)
    -- Ordered list of video labels
    """
    dataDir = os.getcwd()+"/trafficdb/video/"
    dataNameList = os.listdir(dataDir)
    fileCount = 0
    totalFiles = 0
    ImMs=sio.loadmat(os.getcwd()+'/trafficdb/ImageMaster.mat')        
    class_data = []
    labelList = []
    fname=[]
    totalFiles = np.size(os.listdir(dataDir))
    bigBuffer = np.zeros((totalFiles,numFrames,height,width))
    data_label = []
    #fileCount = 0
    #for filename in os.listdir(dataDir):
    for count in range(254):
        s=ImMs['imagemaster'][0][count][0][0][1][0]
        data_label.append(ImMs['imagemaster'][0][count][0][0][2][0])
        filename=[''.join(s)][0]
        fname.append(filename)
        vid = cv2.VideoCapture(dataDir+"/"+filename+".avi")
        vidFrames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
        buffer = np.zeros((numFrames,height,width))
        for i in range(numFrames):
            sane, thisFrame = vid.read()                
            if not sane:
                vid.set(cv2.CAP_PROP_POS_FRAMES, 0)
                sane, thisFrame = vid.read()
                if not sane:
                    #print("something broke at",i,filename)
                    break
                
            scale = 1
            height2, width2, channels = thisFrame.shape
            centerX,centerY=int(height2/2),int(width2/2)
            radiusX,radiusY = int(centerX*scale), int(centerY*scale)
            minX,maxX=centerX-radiusX,centerX+radiusX
            minY,maxY=centerY-radiusY,centerY+radiusY
            cropped = thisFrame[minX:maxX, minY:maxY]
                
            buffer[i] = cv2.cvtColor(cv2.resize(cropped,(width,height)),cv2.COLOR_BGR2GRAY)
                
            #label.append(foldername)
        bigBuffer[count]=buffer
        #bigBuffer[fileCount] = buffer
        #fileCount += 1
        vid.release()
    class_data.append(bigBuffer)
        #labelList.append(label)    
    return class_data[0],data_label,fname

trafficdata, traffic_class, filename = import_traffic()

In [ ]:
def predict(X_test,y_test,classifier):
    pred = classifier.predict(X_test)
    acc,per_class = compute_accuracy(pred,y_test)
    return pred,acc,per_class

In [ ]:
def compute_accuracy(pred,y_test):
    assert len(pred) == len(y_test)
    matrix = confusion_matrix(y_test, pred)
    return (np.sum(np.array(pred) == np.array(y_test)) / len(pred)) * 100,matrix.diagonal()/matrix.sum(axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(trafficdata,traffic_class,test_size = 0.4)

In [ ]:
classifier = PM1NN()
classifier.fit(X_train,y_train)

In [ ]:
%%time
pred,acc,per_class = predict(X_test,y_test,classifier)

In [ ]:
print(acc,per_class)